# Leaflet length:width Ratios

In [97]:
import cv2
import numpy as np
import pandas as pd

def calculate_leaflet_properties(line_id, dpi):
    
    # Intialize the image path.
    image_path = f'/Users/brandonulin/Downloads/Hognut/SCANS_15JUL23/LEAVES/{line_id}.jpeg' # note chnage the file extension once we have the scanner figured out.
    
    # Load the image
    image = cv2.imread(image_path)  # Load the image using cv2.imread

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert the image to grayscale using cv2.cvtColor

    # Invert the grayscale image
    inverted = cv2.bitwise_not(gray)  # Invert the grayscale image using cv2.bitwise_not

    # Binarize the inverted image
    _, threshold = cv2.threshold(inverted, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # Binarize the inverted image using cv2.threshold

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # Find contours in the thresholded image using cv2.findContours

    # Sort the contours by area in descending order
    contours = sorted(contours, key=cv2.contourArea, reverse=True)  # Sort the contours by area in descending order using sorted

    # Intialize a dictionary to store the leaflet information.
    df_dicc = {
        'line_id': [line_id, line_id, line_id],
        'dpi': [dpi, dpi, dpi],
        'leaflet_id': [],
        'width': [], 'height': [],
        'ratio': [], 'area': [],
        
    }
    
    # For every leaflet.
    for i, contour in enumerate(contours[:3]):  # Process the three largest contours (assuming they are the leaflets)
        
        # Find the widest points of the leaflet
        widest_points = contour[contour[:, :, 0].argmin()][0], contour[contour[:, :, 0].argmax()][0]  # Find the widest points of the leaflet by finding the minimum and maximum x-coordinate

        # Find the lowest and highest points of the leaflet
        highest_point = contour[contour[:, :, 1].argmax()][0]  # Find the lowest point of the leaflet by finding the maximum y-coordinate
        lowest_point = contour[contour[:, :, 1].argmin()][0]  # Find the highest point of the leaflet by finding the minimum y-coordinate

        # Calculate the centroid of the leaflet
        centroid_x = int((widest_points[0][0] + widest_points[1][0]) / 2)
        centroid_y = int((highest_point[1] + lowest_point[1]) / 2)
        centroid = (centroid_x, centroid_y)

        # Draw the width line
        cv2.line(threshold, (widest_points[0][0], widest_points[0][1]), (widest_points[1][0], widest_points[1][1]), (0, 0, 255), 2)  # Draw the width line on the thresholded image using cv2.line

        # Draw the height line
        cv2.line(threshold, (lowest_point[0], lowest_point[1]), (highest_point[0], highest_point[1]), (0, 0, 255), 2)  # Draw the height line on the thresholded image using cv2.line

        # Calculate the properties of the leaflet
        leaflet_height = abs(highest_point[1] - lowest_point[1])  # Calculate the height of the leaflet by taking the absolute difference in y-coordinates
        leaflet_width = abs(widest_points[1][0] - widest_points[0][0])  # Calculate the width of the leaflet by taking the absolute difference in x-coordinates
        leaflet_area = cv2.contourArea(contour)  # Calculate the area of the leaflet using cv2.contourArea
        leaflet_ratio = leaflet_width / leaflet_height  # Calculate the aspect ratio of the leaflet
        
        # Append the dictionary with the leaflet information.
        df_dicc['leaflet_id'].append(i + 1)
        df_dicc['height'].append(leaflet_height)
        df_dicc['width'].append(leaflet_width)
        df_dicc['ratio'].append(leaflet_ratio)
        df_dicc['area'].append(leaflet_area)

        # Add height and width labels
        height_y = centroid_y + ((centroid_y - lowest_point[1]) // 5)
        width_y = centroid_y + ((centroid_y - lowest_point[1]) // 10)
        cv2.putText(threshold, f"Height: {leaflet_height}", (centroid_x, height_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)  # Add height value on the line
        cv2.putText(threshold, f"Width: {leaflet_width}", (centroid_x, width_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)  # Add width value on the line

        # Add leaflet label at the centroid
        cv2.putText(threshold, "Leaflet " + str(i + 1), centroid, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # Add the leaflet label using cv2.putText

    # Intialize the export path.
    export_path = f'/Users/brandonulin/Downloads/Hognut/SCANS_PROCESSED/{line_id}'
    
    # Convert the dictionary to a pandas dataframe and export.
    df = pd.DataFrame(df_dicc)
    df.to_csv(export_path+'.csv', index=False)
    
    # Save the processed image.
    cv2.imwrite(export_path+'.jpg', threshold)  # Save the thresholded image

    cv2.destroyAllWindows()
    return

In [99]:
calculate_leaflet_properties('BP3B_L',500)

In [18]:
df= pd.read_csv('/Users/brandonulin/Downloads/Hognut/SCANS_PROCESSED/LG25C_L.csv')

In [19]:
df

,line_id,dpi,leaflet_id,width,height,ratio,area
0,LG25C_L,500,1,964,1158,0.832470,698225.0
1,LG25C_L,500,2,812,1089,0.745638,583475.5
2,LG25C_L,500,3,808,1054,0.766603,556333.5


## df
# Save to csv.

In [ ]:
# import cv2
# import numpy as np
# import os

# def calculate_leaflet_properties(image_path):
#     # Load the image
#     image = cv2.imread(image_path)  # Load the image using cv2.imread

#     # Convert the image to grayscale
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert the image to grayscale using cv2.cvtColor

#     # Invert the grayscale image
#     inverted = cv2.bitwise_not(gray)  # Invert the grayscale image using cv2.bitwise_not

#     # Binarize the inverted image
#     _, threshold = cv2.threshold(inverted, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # Binarize the inverted image using cv2.threshold

#     # Find contours in the thresholded image
#     contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # Find contours in the thresholded image using cv2.findContours

#     # Sort the contours by area in descending order
#     contours = sorted(contours, key=cv2.contourArea, reverse=True)  # Sort the contours by area in descending order using sorted

#     for i, contour in enumerate(contours[:3]):  # Process the three largest contours (assuming they are the leaflets)
#         # Find the widest points of the leaflet
#         widest_points = contour[contour[:, :, 0].argmin()][0], contour[contour[:, :, 0].argmax()][0]  # Find the widest points of the leaflet by finding the minimum and maximum x-coordinate

#         # Find the lowest and highest points of the leaflet
#         highest_point = contour[contour[:, :, 1].argmax()][0]  # Find the lowest point of the leaflet by finding the maximum y-coordinate
#         lowest_point = contour[contour[:, :, 1].argmin()][0]  # Find the highest point of the leaflet by finding the minimum y-coordinate

#         # Calculate the centroid of the leaflet
#         centroid_x = int((widest_points[0][0] + widest_points[1][0]) / 2)
#         centroid_y = int((highest_point[1] + lowest_point[1]) / 2)
#         centroid = (centroid_x, centroid_y)

#         # Draw the width line
#         cv2.line(threshold, (widest_points[0][0], widest_points[0][1]), (widest_points[1][0], widest_points[1][1]), (0, 0, 255), 2)  # Draw the width line on the thresholded image using cv2.line

#         # Draw the height line
#         cv2.line(threshold, (lowest_point[0], lowest_point[1]), (highest_point[0], highest_point[1]), (0, 0, 255), 2)  # Draw the height line on the thresholded image using cv2.line

#         # Calculate the properties of the leaflet
#         leaflet_height = abs(highest_point[1] - lowest_point[1])  # Calculate the height of the leaflet by taking the absolute difference in y-coordinates
#         leaflet_width = abs(widest_points[1][0] - widest_points[0][0])  # Calculate the width of the leaflet by taking the absolute difference in x-coordinates
#         leaflet_area = cv2.contourArea(contour)  # Calculate the area of the leaflet using cv2.contourArea
#         leaflet_ratio = leaflet_width / leaflet_height  # Calculate the aspect ratio of the leaflet

        

#         # Add height and width labels
#         height_y = centroid_y + ((centroid_y - lowest_point[1]) // 5)
#         width_y = centroid_y + ((centroid_y - lowest_point[1]) // 10)
        
#         # Print the calculated properties
#         print("Leaflet", i + 1)  # Print the label of the leaflet
#         print("Height:", leaflet_height)  # Print the height of the leaflet
#         print("Width:", leaflet_width)  # Print the width of the leaflet
#         print("Area:", leaflet_area)  # Print the area of the leaflet
#         print("Aspect Ratio:", leaflet_ratio)  # Print the aspect ratio of the leaflet
#         print('c-coords', centroid)
#         print('h-coords', (centroid_x, height_y))
#         print('w-coords', (centroid_x, width_y))
#         print()
        
#         cv2.putText(threshold, f"Height: {leaflet_height}", (centroid_x, height_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)  # Add height value on the line
#         cv2.putText(threshold, f"Width: {leaflet_width}", (centroid_x, width_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)  # Add width value on the line

#         # Add leaflet label at the centroid
#         cv2.putText(threshold, "Leaflet " + str(i + 1), centroid, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # Add the leaflet label using cv2.putText

#     cv2.imshow("Leaflets", threshold)  # Display the thresholded image with the width and height lines using cv2.imshow
    
#     #Save the processed image
#     processed_folder = '/Users/brandonulin/Downloads/Hognut/Processed Images/'
#     processed_image_path = os.path.join(processed_folder, os.path.basename(image_path))
#     cv2.imwrite(processed_image_path, threshold)  # Save the thresholded image

#     while True:
#         key = cv2.waitKey(1) & 0xFF
#         if key == ord('q'):
#             break

#     cv2.destroyAllWindows()  # Close all windows

# def main():
#     base_folder = '/Users/brandonulin/Downloads/Hognut/test_scan/'  # Specify the base folder path

#     while True:
#         # Input the image name
#         image_name = input("Enter the image name (or 'q' to quit): ")
#         if image_name.lower() == 'q':
#             break

#         image_path = os.path.join(base_folder, image_name)  # Construct the full image path
#         if not os.path.isfile(image_path):
#             print("Invalid image name. Please try again.")
#             continue

#         calculate_leaflet_properties(image_path)

# # Call the main function to start the code
# main()


In [12]:
# df

,line_id,leaflet_id,width,height,ratio,area
0,Leaf_cut_3leaflets_test,1,420,510,0.823529,132827.0
1,Leaf_cut_3leaflets_test,2,360,497,0.724346,119500.0
2,Leaf_cut_3leaflets_test,3,352,478,0.736402,109486.5


In [13]:
# width

(array([ 88, 624], dtype=int32), array([440, 645], dtype=int32))

In [4]:
# height[0]

array([244, 305], dtype=int32)

In [15]:
# center

(264, 544)

In [ ]:
h_x = height[0][0] - width[0][0]
h_y = width[0][1] - height[0][1]
w_x = width[1][0] - height[0][0]
w_y = width[0][1] - height[0][1]

In [ ]:
# # import cv2
# # import numpy as np
# /Users/brandonulin/Downloads/Hognut/Processed Images/
# # # Path to the input image
# # image_path = '/Users/brandonulin/Downloads/Hognut/test_scan/Scanned_Documents_6_Page_1.jpg'

# # def calculate_leaflet_properties(image_path):
# #     # Load the image
# #     image = cv2.imread(image_path)
    
# #     # Convert the image to grayscale
# #     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
# #     # Apply thresholding to segment the leaflets
# #     _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
# #     # Find contours in the thresholded image
# #     contours, _ = cv2.findContours(threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# #     # Sort the contours by area in descending order
# #     contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
# #     # Process the three largest contours (assuming they are the leaflets)
# #     for i in range(min(3, len(contours))):
# #         # Approximate the contour with a polygon
# #         epsilon = 0.02 * cv2.arcLength(contours[i], True)
# #         approx = cv2.approxPolyDP(contours[i], epsilon, True)

# #         # Calculate the bounding rectangle of the contour
# #         x, y, w, h = cv2.boundingRect(approx)

# #         # Draw the bounding rectangle on the image
# #         cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# #         # Find the widest points of the leaflet
# #         widest_points = approx[approx[:, :, 0].argmin()][0], approx[approx[:, :, 0].argmax()][0]

# #         # Find the lowest and highest points of the leaflet
# #         lowest_point = approx[approx[:, :, 1].argmax()][0]
# #         highest_point = approx[approx[:, :, 1].argmin()][0]
        
# #         # Draw the width line
# #         cv2.line(image, (widest_points[0][0], widest_points[0][1]), (widest_points[1][0], widest_points[1][1]), (255, 0, 0), 2)
        
# #         # Draw the height line
# #         cv2.line(image, (lowest_point[0], lowest_point[1]), (highest_point[0], highest_point[1]), (0, 0, 255), 2)
        
# #         # Calculate the properties of the leaflet
# #         leaflet_height = abs(highest_point[1] - lowest_point[1])
# #         leaflet_width = abs(widest_points[1][0] - widest_points[0][0])
# #         leaflet_area = cv2.contourArea(contours[i])
# #         leaflet_ratio = leaflet_width / leaflet_height
        
# #         # Print the calculated properties
# #         print("Leaflet", i + 1)
# #         print("Height:", leaflet_height)
# #         print("Width:", leaflet_width)
# #         print("Area:", leaflet_area)
# #         print("Aspect Ratio:", leaflet_ratio)
# #         print()
    
# #     # Display the image with bounding rectangles, width lines, and height lines
# #     cv2.imshow("Leaflets", image)
# #     cv2.waitKey(0)
# #     cv2.destroyAllWindows()

# # # Path to the image file
# # image_path = '/Users/brandonulin/Downloads/Hognut/test_scan/Scanned_Documents_6_Page_1.jpg'

# # # Call the function to calculate leaflet properties
# # calculate_leaflet_properties(image_path)

Leaflet 1
Height: 1193
Width: 1017
Area: 399251.5
Aspect Ratio: 0.8524727577535625

Leaflet 2
Height: 25
Width: 30
Area: 46.5
Aspect Ratio: 1.2

Leaflet 3
Height: 10
Width: 5
Area: 33.0
Aspect Ratio: 0.5



In [ ]:
# import cv2
# import numpy as np

# # Path to the input image
# image_path = '/Users/brandonulin/Downloads/Hognut/test_scan/Scanned_Documents_6_Page_2.jpg'

# # Load the image
# image = cv2.imread(image_path)

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to obtain a binary image with white edges
# _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# # Find contours in the thresholded image
# contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Find the largest contour (assumed to be the leaf)
# largest_contour = max(contours, key=cv2.contourArea)

# # Create a blank black image of the same size as the original image
# result = np.zeros_like(image)

# # Draw the largest contour on the result image as white
# cv2.drawContours(result, [largest_contour], 0, (255, 255, 255), thickness=cv2.FILLED)

# # Convert the result image to grayscale
# result_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to create a binary image with white leaf and black background
# _, result_threshold = cv2.threshold(result_gray, 0, 255, cv2.THRESH_BINARY)

# # Find contours in the binary image to obtain the dimensions of the leaf
# leaf_contours, _ = cv2.findContours(result_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# leaf_contour = max(leaf_contours, key=cv2.contourArea)
# leaf_x, leaf_y, leaf_w, leaf_h = cv2.boundingRect(leaf_contour)

# # Find the two widest points on the leaf contour
# left_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmin()][0])
# right_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmax()][0])

# # Find the top and bottom points on the leaf contour
# top_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmin()][0])
# bottom_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmax()][0])

# # Calculate the midpoints along the x-axis and y-axis
# midpoint_x = int((left_point[0] + right_point[0]) / 2)
# midpoint_y = int((top_point[1] + bottom_point[1]) / 2)

# # Find the central vein of the leaf contour
# hierarchy = cv2.findContours(result_threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-1]
# vein_contour = None
# for i in range(len(hierarchy[0])):
#     if hierarchy[0][i][3] == 0:
#         vein_contour = leaf_contours[i]
#         break

# # Calculate the leaf area
# leaf_area = cv2.contourArea(largest_contour)

# # Calculate the ratio between width and height
# leaf_width = abs(left_point[0] - right_point[0])
# leaf_height = abs(top_point[1] - bottom_point[1])
# leaf_ratio = leaf_width / leaf_height

# # Remove the white background from the image
# image[result_threshold == 0] = [0, 0, 0]

# # Draw lines for width, height, vein, and dimensions
# cv2.line(image, left_point, right_point, (0, 255, 0), thickness=2)
# cv2.putText(image, f"Width: {leaf_width}", (midpoint_x, midpoint_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# cv2.line(image, top_point, bottom_point, (0, 0, 255), thickness=2)
# cv2.putText(image, f"Height: {leaf_height}", (midpoint_x, midpoint_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# cv2.polylines(image, [vein_contour], False, (0, 0, 255), thickness=2)

# # Print the dimensions, area, and ratio of the leaf
# print("Leaf Width:", leaf_width)
# print("Leaf Height:", leaf_height)
# print("Leaf Area:", leaf_area)
# print("Leaf Ratio:", leaf_ratio)

# # Crop the image around the leaf
# cropped_image = image[leaf_y:leaf_y + leaf_h, leaf_x:leaf_x + leaf_w]

# # Display the cropped image
# cv2.imshow("Cropped Image", cropped_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


Leaf Width: 506
Leaf Height: 632
Leaf Area: 190013.0
Leaf Ratio: 0.8006329113924051


In [ ]:
# import cv2
# import numpy as np

# # Path to the input image
# image_path = '/Users/brandonulin/Downloads/Hognut/test_scan/Scanned_Documents_6_Page_2.jpg'

# # Load the image
# image = cv2.imread(image_path)

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to obtain a binary image with white edges
# _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# # Find contours in the thresholded image
# contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Find the largest contour (assumed to be the leaf)
# largest_contour = max(contours, key=cv2.contourArea)

# # Create a blank black image of the same size as the original image
# result = np.zeros_like(image)

# # Draw the largest contour on the result image as white
# cv2.drawContours(result, [largest_contour], 0, (255, 255, 255), thickness=cv2.FILLED)

# # Convert the result image to grayscale
# result_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to create a binary image with white leaf and black background
# _, result_threshold = cv2.threshold(result_gray, 0, 255, cv2.THRESH_BINARY)

# # Find contours in the binary image to obtain the dimensions of the leaf
# leaf_contours, _ = cv2.findContours(result_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# leaf_contour = max(leaf_contours, key=cv2.contourArea)
# leaf_x, leaf_y, leaf_w, leaf_h = cv2.boundingRect(leaf_contour)

# # Find the two widest points on the leaf contour
# left_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmin()][0])
# right_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmax()][0])

# # Find the top and bottom points on the leaf contour
# top_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmin()][0])
# bottom_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmax()][0])

# # Calculate the midpoints along the x-axis and y-axis
# midpoint_x = int((left_point[0] + right_point[0]) / 2)
# midpoint_y = int((top_point[1] + bottom_point[1]) / 2)

# # Find the central vein of the leaf contour
# hierarchy = cv2.findContours(result_threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-1]
# vein_contour = None
# for i in range(len(hierarchy[0])):
#     if hierarchy[0][i][3] == 0:
#         vein_contour = leaf_contours[i]
#         break

# # Calculate the leaf area
# leaf_area = cv2.contourArea(largest_contour)

# # Calculate the ratio between width and height
# leaf_width = abs(left_point[0] - right_point[0])
# leaf_height = abs(top_point[1] - bottom_point[1])
# leaf_ratio = leaf_width / leaf_height

# # Remove the white background from the image
# image[result_threshold == 0] = [0, 0, 0]

# # Draw linesfor width, height, vein, and dimensions
# cv2.line(image, left_point, right_point, (0, 255, 0), thickness=2)
# cv2.line(image, top_point, bottom_point, (0, 0, 255), thickness=2)
# cv2.polylines(image, [vein_contour], False, (0, 0, 255), thickness=2)

# # Print the dimensions, area, and ratio of the leaf
# print("Leaf Width:", leaf_width)
# print("Leaf Height:", leaf_height)
# print("Leaf Area:", leaf_area)
# print("Leaf Ratio:", leaf_ratio)

# # Display the result image with the lines and dimensions
# cv2.imshow("Leaf Image", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


Leaf Width: 506
Leaf Height: 632
Leaf Area: 190013.0
Leaf Ratio: 0.8006329113924051


In [ ]:
# import cv2
# import numpy as np

# # Path to the input image
# image_path = '/Users/brandonulin/Downloads/Hognut/test_scan/test_scan5_Page_1.jpg'

# # Load the image
# image = cv2.imread(image_path)

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to obtain a binary image with white edges
# _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# # Find contours in the thresholded image
# contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Find the largest contour (assumed to be the leaf)
# largest_contour = max(contours, key=cv2.contourArea)

# # Create a blank black image of the same size as the original image
# result = np.zeros_like(image)

# # Draw the largest contour on the result image as white
# cv2.drawContours(result, [largest_contour], 0, (255, 255, 255), thickness=cv2.FILLED)

# # Convert the result image to grayscale
# result_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to create a binary image with white leaf and black background
# _, result_threshold = cv2.threshold(result_gray, 0, 255, cv2.THRESH_BINARY)

# # Find contours in the binary image to obtain the dimensions of the leaf
# leaf_contours, _ = cv2.findContours(result_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# leaf_contour = max(leaf_contours, key=cv2.contourArea)
# leaf_x, leaf_y, leaf_w, leaf_h = cv2.boundingRect(leaf_contour)

# # Crop the image to the bounding rectangle of the leaf
# cropped_image = image[leaf_y:leaf_y + leaf_h, leaf_x:leaf_x + leaf_w]

# # Find the two widest points on the leaf contour
# left_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmin()][0])
# right_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmax()][0])

# # Find the top and bottom points on the leaf contour
# top_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmin()][0])
# bottom_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmax()][0])

# # Calculate the width and height
# leaf_width = abs(left_point[0] - right_point[0])
# leaf_height = abs(top_point[1] - bottom_point[1])

# # Calculate the area
# leaf_area = leaf_width * leaf_height

# # Calculate the ratio between width and height
# leaf_ratio = leaf_width / leaf_height

# # Find the new coordinates of the points and contours after cropping
# left_point_cropped = (left_point[0] - leaf_x, left_point[1] - leaf_y)
# right_point_cropped = (right_point[0] - leaf_x, right_point[1] - leaf_y)
# top_point_cropped = (top_point[0] - leaf_x, top_point[1] - leaf_y)
# bottom_point_cropped = (bottom_point[0] - leaf_x, bottom_point[1] - leaf_y)

# # Create the points for the width line
# width_line_points = np.array([left_point_cropped, right_point_cropped], dtype=np.int32)

# # Create the points for the height line
# height_line_points = np.array([top_point_cropped, bottom_point_cropped], dtype=np.int32)

# # Calculate the angle of the height line
# angle = np.arctan2(bottom_point_cropped[1] - top_point_cropped[1], bottom_point_cropped[0] - top_point_cropped[0])
# angle_degrees = np.degrees(angle)

# # Define the position of the height text
# height_text_pos = (bottom_point_cropped[0], bottom_point_cropped[1] + leaf_y)

# # Calculate the rotation center as the midpoint of the height line
# rotation_center = ((top_point_cropped[0] + bottom_point_cropped[0]) / 2, (top_point_cropped[1] + bottom_point_cropped[1]) / 2)

# # Rotate the text using the affine transformation
# text_scale = 1  # Adjust the scale factor if needed
# rotation_matrix = cv2.getRotationMatrix2D(rotation_center, angle_degrees, text_scale)

# # Define the text position as a column vector [x, y, 1]
# height_text_pos_homogeneous = np.array([height_text_pos[0], height_text_pos[1], 1]).reshape((3, 1))

# # Perform matrix multiplication to obtain the rotated position
# height_text_pos_rotated_homogeneous = rotation_matrix.dot(height_text_pos_homogeneous)
# height_text_pos_rotated = (height_text_pos_rotated_homogeneous[0, 0], height_text_pos_rotated_homogeneous[1, 0])

# # Convert the rotated text position back to integers
# height_text_pos_rotated = (int(height_text_pos_rotated[0]), int(height_text_pos_rotated[1]))

# # Create the points for the height line
# height_line_points = np.array([top_point_cropped, bottom_point_cropped], dtype=np.int32)

# # Add the height text position to the height line points
# height_line_points = np.vstack((height_line_points, height_text_pos_rotated))

# # Draw lines for width and height on the cropped image with different colors
# cv2.drawContours(cropped_image, [width_line_points], -1, (255, 0, 0), thickness=4)  # Blue color for width line
# cv2.drawContours(cropped_image, [height_line_points], -1, (0, 0, 255), thickness=2)  # Red color for height line

# # Calculate the midpoint of the height line
# height_text_pos_rotated = height_line_points[int(len(height_line_points) / 2)]

# # Define the text color
# text_color = (255, 255, 255)

# # Draw the height text at the updated position
# cv2.putText(cropped_image, f'Height: {leaf_height}', tuple(height_text_pos_rotated), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)

# # Add text for width and rotated height on the cropped image, positioned on the lines
# width_text_pos = (int((left_point_cropped[0] + right_point_cropped[0]) / 2), int((left_point_cropped[1] + right_point_cropped[1]) / 2))

# # Set the text color to white
# text_color = (255, 255, 255)

# cv2.putText(cropped_image, f'Width: {leaf_width}', width_text_pos, cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)
# cv2.putText(cropped_image, f'Height: {leaf_height}', height_text_pos_rotated, cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)

# # Print the dimensions, area, and ratio of the leaf
# print("Leaf Width:", leaf_width)
# print("Leaf Height:", leaf_height)
# print("Leaf Area:", leaf_area)
# print("Leaf Ratio:", leaf_ratio)

# # Display the cropped image with the lines and dimensions
# cv2.imshow("Cropped Leaf Image", cropped_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


Leaf Width: 767
Leaf Height: 878
Leaf Area: 673426
Leaf Ratio: 0.8735763097949886


In [ ]:
# import cv2
# import numpy as np

# # Path to the input image
# image_path = '/Users/brandonulin/Downloads/Hognut/test_scan/test_scan5_Page_1.jpg'

# # Load the image
# image = cv2.imread(image_path)

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to obtain a binary image with white edges
# _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# # Find contours in the thresholded image
# contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Find the largest contour (assumed to be the leaf)
# largest_contour = max(contours, key=cv2.contourArea)

# # Create a blank black image of the same size as the original image
# result = np.zeros_like(image)

# # Draw the largest contour on the result image as white
# cv2.drawContours(result, [largest_contour], 0, (255, 255, 255), thickness=cv2.FILLED)

# # Convert the result image to grayscale
# result_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to create a binary image with white leaf and black background
# _, result_threshold = cv2.threshold(result_gray, 0, 255, cv2.THRESH_BINARY)

# # Find contours in the binary image to obtain the dimensions of the leaf
# leaf_contours, _ = cv2.findContours(result_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# leaf_contour = max(leaf_contours, key=cv2.contourArea)
# leaf_x, leaf_y, leaf_w, leaf_h = cv2.boundingRect(leaf_contour)

# # Find the two widest points on the leaf contour
# left_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmin()][0])
# right_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmax()][0])

# # Find the top and bottom points on the leaf contour
# top_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmin()][0])
# bottom_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmax()][0])

# # Calculate the midpoints along the x-axis and y-axis
# midpoint_x = int((left_point[0] + right_point[0]) / 2)
# midpoint_y = int((top_point[1] + bottom_point[1]) / 2)

# # Find the central vein of the leaf contour
# hierarchy = cv2.findContours(result_threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-1]
# vein_contour = None
# for i in range(len(hierarchy[0])):
#     if hierarchy[0][i][3] == 0:
#         vein_contour = leaf_contours[i]
#         break

# # Calculate the leaf area
# leaf_area = cv2.contourArea(largest_contour)

# # Remove the white background from the image
# image[result_threshold == 0] = [0, 0, 0]

# # Draw lines for width, height, and vein
# cv2.line(image, left_point, right_point, (0, 255, 0), thickness=2)
# cv2.line(image, top_point, bottom_point, (0, 0, 255), thickness=2)
# cv2.polylines(image, [vein_contour], False, (0, 0, 255), thickness=2)

# # Print the dimensions and area of the leaf
# print("Leaf Width:", abs(left_point[0] - right_point[0]))
# print("Leaf Height:", abs(top_point[1] - bottom_point[1]))
# print("Leaf Area:", leaf_area)

# # Display the result image with the lines and dimensions
# cv2.imshow("Leaf Image", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


Leaf Width: 767
Leaf Height: 878
Leaf Area: 412001.5


In [ ]:
# import cv2
# import numpy as np

# # Path to the input image
# image_path = '/Users/brandonulin/Downloads/Hognut/test_scan/test_scan5_Page_1.jpg'

# # Load the image
# image = cv2.imread(image_path)

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to obtain a binary image with white edges
# _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# # Find contours in the thresholded image
# contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Find the largest contour (assumed to be the leaf)
# largest_contour = max(contours, key=cv2.contourArea)

# # Create a blank black image of the same size as the original image
# result = np.zeros_like(image)

# # Draw the largest contour on the result image as white
# cv2.drawContours(result, [largest_contour], 0, (255, 255, 255), thickness=cv2.FILLED)

# # Convert the result image to grayscale
# result_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to create a binary image with white leaf and black background
# _, result_threshold = cv2.threshold(result_gray, 0, 255, cv2.THRESH_BINARY)

# # Find contours in the binary image to obtain the dimensions of the leaf
# leaf_contours, _ = cv2.findContours(result_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# leaf_contour = max(leaf_contours, key=cv2.contourArea)
# leaf_x, leaf_y, leaf_w, leaf_h = cv2.boundingRect(leaf_contour)

# # Crop the image to the bounding rectangle of the leaf
# cropped_image = image[leaf_y:leaf_y + leaf_h, leaf_x:leaf_x + leaf_w]

# # Find the two widest points on the leaf contour
# left_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmin()][0])
# right_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmax()][0])

# # Find the top and bottom points on the leaf contour
# top_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmin()][0])
# bottom_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmax()][0])

# # Calculate the width and height
# leaf_width = abs(left_point[0] - right_point[0])
# leaf_height = abs(top_point[1] - bottom_point[1])

# # Calculate the ratio between width and height
# leaf_ratio = leaf_width / leaf_height

# # Find the new coordinates of the points and contours after cropping
# left_point_cropped = (left_point[0] - leaf_x, left_point[1] - leaf_y)
# right_point_cropped = (right_point[0] - leaf_x, right_point[1] - leaf_y)
# top_point_cropped = (top_point[0] - leaf_x, top_point[1] - leaf_y)
# bottom_point_cropped = (bottom_point[0] - leaf_x, bottom_point[1] - leaf_y)
# vein_contour_cropped = leaf_contour - (leaf_x, leaf_y)

# # Draw lines for width, height, and vein on the cropped image
# cv2.line
# import cv2
# import numpy as np

# # Path to the input image
# image_path = '/Users/brandonulin/Downloads/Hognut/test_scan/test_scan5_Page_1.jpg'

# # Load the image
# image = cv2.imread(image_path)

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to obtain a binary image with white edges
# _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# # Find contours in the thresholded image
# contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Find the largest contour (assumed to be the leaf)
# largest_contour = max(contours, key=cv2.contourArea)

# # Create a blank black image of the same size as the original image
# result = np.zeros_like(image)

# # Draw the largest contour on the result image as white
# cv2.drawContours(result, [largest_contour], 0, (255, 255, 255), thickness=cv2.FILLED)

# # Convert the result image to grayscale
# result_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to create a binary image with white leaf and black background
# _, result_threshold = cv2.threshold(result_gray, 0, 255, cv2.THRESH_BINARY)

# # Find contours in the binary image to obtain the dimensions of the leaf
# leaf_contours, _ = cv2.findContours(result_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# leaf_contour = max(leaf_contours, key=cv2.contourArea)
# leaf_x, leaf_y, leaf_w, leaf_h = cv2.boundingRect(leaf_contour)

# # Crop the image to the bounding rectangle of the leaf
# cropped_image = image[leaf_y:leaf_y + leaf_h, leaf_x:leaf_x + leaf_w]

# # Find the two widest points on the leaf contour
# left_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmin()][0])
# right_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmax()][0])

# # Find the top and bottom points on the leaf contour
# top_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmin()][0])
# bottom_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmax()][0])

# # Calculate the width and height
# leaf_width = abs(left_point[0] - right_point[0])
# leaf_height = abs(top_point[1] - bottom_point[1])

# # Calculate the ratio between width and height
# leaf_ratio = leaf_width / leaf_height

# # Find the new coordinates of the points and contours after cropping
# left_point_cropped = (left_point[0] - leaf_x, left_point[1] - leaf_y)
# right_point_cropped = (right_point[0] - leaf_x, right_point[1] - leaf_y)
# top_point_cropped = (top_point[0] - leaf_x, top_point[1] - leaf_y)
# bottom_point_cropped = (bottom_point[0] - leaf_x, bottom_point[1] - leaf_y)
# vein_contour_cropped = leaf_contour - (leaf_x, leaf_y)

# # Draw lines for width, height, and vein on the cropped image
# cv2.line(cropped_image, top_point_cropped, bottom_point_cropped, (0, 0, 255), thickness=2)
# cv2.polylines(cropped_image, [vein_contour_cropped], False, (0, 0, 255), thickness=2)

# # Display the cropped image with the lines and dimensions
# cv2.imshow("Cropped Leaf Image", cropped_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [1]:
# import cv2
# import numpy as np

# # Global variables
# drawing = False
# points = []

# def mouse_callback(event, x, y, flags, param):
#     global drawing, points

#     # Check if the event is a left mouse button click
#     if event == cv2.EVENT_LBUTTONDOWN:
#         drawing = True
#         points = [(x, y)]

#     # Check if the event is mouse movement with the left button pressed
#     elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
#         if drawing:
#             points.append((x, y))

#     # Check if the event is a left mouse button release
#     elif event == cv2.EVENT_LBUTTONUP:
#         drawing = False
#         cv2.line(image, points[0], points[-1], (0, 0, 255), thickness=2)
#         cv2.imshow("Leaf Image", image)


# # Path to the input image
# image_path = '/Users/brandonulin/Downloads/Hognut/test_scan/test1_test.jpg'

# # Load the image
# image = cv2.imread(image_path)

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to obtain a binary image with white edges
# _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# # Find contours in the thresholded image
# contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Find the largest contour (assumed to be the leaf)
# largest_contour = max(contours, key=cv2.contourArea)

# # Create a blank black image of the same size as the original image
# result = np.zeros_like(image)

# # Draw the largest contour on the result image as white
# cv2.drawContours(result, [largest_contour], 0, (255, 255, 255), thickness=cv2.FILLED)

# # Convert the result image to grayscale
# result_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to create a binary image with white leaf and black background
# _, result_threshold = cv2.threshold(result_gray, 0, 255, cv2.THRESH_BINARY)

# # Find contours in the binary image to obtain the dimensions of the leaf
# leaf_contours, _ = cv2.findContours(result_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# leaf_contour = max(leaf_contours, key=cv2.contourArea)
# leaf_x, leaf_y, leaf_w, leaf_h = cv2.boundingRect(leaf_contour)

# # Find the two widest points on the leaf contour
# left_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmin()][0])
# right_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmax()][0])

# # Calculate the midpoints along the x-axis and y-axis
# midpoint_x = int((left_point[0] + right_point[0]) / 2)
# midpoint_y = int((leaf_y + leaf_y + leaf_h) / 2)

# # Find the central vein of the leaf contour
# hierarchy = cv2.findContours(result_threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-1]
# vein_contour = None
# for i in range(len(hierarchy[0])):
#     if hierarchy[0][i][3] == 0:
#         vein_contour = leaf_contours[i]
#         break

# # Calculate the leaf area
# leaf_area = cv2.contourArea(largest_contour)

# # Remove the white background from the image
# image[result_threshold == 0] = [0, 0, 0]

# # Create a window and set the mouse callback function
# cv2.namedWindow("Leaf Image")
# cv2.setMouseCallback("Leaf Image", mouse_callback)

# # Display the initial image
# cv2.imshow("Leaf Image", image)

# while True:
#     # Check for key press
#     key = cv2.waitKey(1) & 0xFF

#     # Exit the program when 'q' is pressed
#     if key == ord('q'):
#         break

# cv2.destroyAllWindows()


In [ ]:
# import cv2
# import numpy as np

# # Path to the input image
# image_path = '/Users/brandonulin/Downloads/Hognut/test_scan/test1_test.jpg'

# # Load the image
# image = cv2.imread(image_path)

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to obtain a binary image with white edges
# _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# # Find contours in the thresholded image
# contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Find the largest contour (assumed to be the leaf)
# largest_contour = max(contours, key=cv2.contourArea)

# # Create a blank black image of the same size as the original image
# result = np.zeros_like(image)

# # Draw the largest contour on the result image as white
# cv2.drawContours(result, [largest_contour], 0, (255, 255, 255), thickness=cv2.FILLED)

# # Convert the result image to grayscale
# result_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to create a binary image with white leaf and black background
# _, result_threshold = cv2.threshold(result_gray, 0, 255, cv2.THRESH_BINARY)

# # Find contours in the binary image to obtain the dimensions of the leaf
# leaf_contours, _ = cv2.findContours(result_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# leaf_contour = max(leaf_contours, key=cv2.contourArea)
# leaf_x, leaf_y, leaf_w, leaf_h = cv2.boundingRect(leaf_contour)

# # Find the two widest points on the leaf contour
# left_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmin()][0])
# right_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmax()][0])

# # Find the top and bottom points on the leaf contour
# top_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmin()][0])
# bottom_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmax()][0])

# # Calculate the midpoints along the x-axis and y-axis
# midpoint_x = int((left_point[0] + right_point[0]) / 2)
# midpoint_y = int((top_point[1] + bottom_point[1]) / 2)

# # Find the central vein of the leaf contour
# hierarchy = cv2.findContours(result_threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-1]
# vein_contour = None
# for i in range(len(hierarchy[0])):
#     if hierarchy[0][i][3] == 0:
#         vein_contour = leaf_contours[i]
#         break

# # Draw lines for width, height, and vein
# cv2.line(image, left_point, right_point, (0, 255, 0), thickness=2)
# cv2.line(image, top_point, bottom_point, (0, 0, 255), thickness=2)
# cv2.polylines(image, [vein_contour], False, (0, 0, 255), thickness=2)

# # Calculate the width between the two widest points
# width = abs(left_point[0] - right_point[0])

# # Calculate the height between the top and bottom points
# height = abs(top_point[1] - bottom_point[1])

# # Print the dimensions of the leaf
# print("Leaf Width:", width)
# print("Leaf Height:", height)

# # Display the result image with the lines and dimensions
# cv2.imshow("Leaf Image", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Leaf Width: 693
Leaf Height: 856


In [1]:
# import cv2
# import numpy as np

# # Global variables
# zoomed_image = None
# zoom_factor = 1.0
# drawing = False
# points = []

# def mouse_callback(event, x, y, flags, param):
#     global zoomed_image, zoom_factor, drawing, points

#     # Check if the event is a mouse wheel scroll
#     if event == cv2.EVENT_MOUSEWHEEL:
#         # Update the zoom factor based on the scroll direction
#         if flags > 0:
#             zoom_factor += 0.1
#         else:
#             zoom_factor -= 0.1
#         # Ensure the zoom factor is within a valid range
#         zoom_factor = max(0.1, zoom_factor)
#         # Apply the zoom factor and display the zoomed image
#         h, w = image.shape[:2]
#         zoomed_width = int(w * zoom_factor)
#         zoomed_height = int(h * zoom_factor)
#         zoomed_image = cv2.resize(image.copy(), (zoomed_width, zoomed_height), interpolation=cv2.INTER_LINEAR)
#         cv2.imshow("Leaf Image", zoomed_image)

#     # Check if the event is a left mouse button click
#     elif event == cv2.EVENT_LBUTTONDOWN:
#         drawing = True
#         points = [(x, y)]

#     # Check if the event is mouse movement with the left button pressed
#     elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
#         if drawing:
#             points.append((x, y))

#     # Check if the event is a left mouse button release
#     elif event == cv2.EVENT_LBUTTONUP:
#         drawing = False
#         # Draw the line on the original image
#         cv2.line(image, points[0], points[-1], (0, 0, 255), thickness=2)
#         # Reset the zoomed image and zoom factor
#         zoomed_image = image.copy()
#         zoom_factor = 1.0
#         cv2.imshow("Leaf Image", zoomed_image)


# # Path to the input image
# image_path = '/Users/brandonulin/Downloads/Hognut/test_scan/test1_test.jpg'

# # Load the image
# image = cv2.imread(image_path)

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to obtain a binary image with white edges
# _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# # Find contours in the thresholded image
# contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Find the largest contour (assumed to be the leaf)
# largest_contour = max(contours, key=cv2.contourArea)

# # Create a blank black image of the same size as the original image
# result = np.zeros_like(image)

# # Draw the largest contour on the result image as white
# cv2.drawContours(result, [largest_contour], 0, (255, 255, 255), thickness=cv2.FILLED)

# # Convert the result image to grayscale
# result_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to create a binary image with white leaf and black background
# _, result_threshold = cv2.threshold(result_gray, 0, 255, cv2.THRESH_BINARY)

# # Find contours in the binary image to obtain the dimensions of the leaf
# leaf_contours, _ = cv2.findContours(result_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# leaf_contour = max(leaf_contours, key=cv2.contourArea)
# leaf_x, leaf_y, leaf_w, leaf_h = cv2.boundingRect(leaf_contour)

# # Find the two widest points on the leaf contour
# left_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmin()][0])
# right_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmax()][0])

# # Find the top and bottom points on the leaf contour
# top_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmin()][0])
# bottom_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmax()][0])

# # Calculate the midpoints along the x-axis and y-axis
# midpoint_x = int((left_point[0] + right_point[0]) / 2)
# midpoint_y = int((top_point[1] + bottom_point[1]) / 2)

# # Find the central vein of the leaf contour
# hierarchy = cv2.findContours(result_threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-1]
# vein_contour = None
# for i in range(len(hierarchy[0])):
#     if hierarchy[0][i][3] == 0:
#         vein_contour = leaf_contours[i]
#         break

# # Calculate the leaf area
# leaf_area = cv2.contourArea(largest_contour)

# # Remove the white background from the image
# image[result_threshold == 0] = [0, 0, 0]

# # Create a zoomed version of the image
# zoomed_image = image.copy()

# # Create a window and set the mouse callback function
# cv2.namedWindow("Leaf Image")
# cv2.setMouseCallback("Leaf Image", mouse_callback)

# # Display the initial image
# cv2.imshow("Leaf Image", zoomed_image)

# while True:
#     # Check for key press
#     key = cv2.waitKey(1) & 0xFF

#     # Exit the program when 'q' is pressed
#     if key == ord('q'):
#         break

# cv2.destroyAllWindows()


In [ ]:
# import cv2
# import numpy as np

# # Path to the input image
# image_path = '/Users/brandonulin/Downloads/Hognut/test_scan/test1.jpeg'

# # Load the image
# image = cv2.imread(image_path)

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to obtain a binary image
# _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# # Find contours in the thresholded image
# contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Find the largest contour (assumed to be the leaf)
# largest_contour = max(contours, key=cv2.contourArea)

# # Get the bounding rectangle of the largest contour
# x, y, w, h = cv2.boundingRect(largest_contour)

# # Print the dimensions of the leaf
# print("Leaf Width:", w)
# print("Leaf Height:", h)

# # Draw the bounding rectangle on the image
# cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# # Display the image with the bounding rectangle
# cv2.imshow("Leaf Image", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


Leaf Width: 679
Leaf Height: 834


In [ ]:
# import cv2
# import numpy as np

# # Path to the input image
# image_path = '/Users/brandonulin/Downloads/Hognut/test_scan/test1.jpeg'

# # Load the image
# image = cv2.imread(image_path)

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to obtain a binary image with white edges
# _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# # Find contours in the thresholded image
# contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Find the largest contour (assumed to be the leaf)
# largest_contour = max(contours, key=cv2.contourArea)

# # Create a blank black image of the same size as the original image
# result = np.zeros_like(image)

# # Draw the largest contour on the result image as white
# cv2.drawContours(result, [largest_contour], 0, (255, 255, 255), thickness=cv2.FILLED)

# # Convert the result image to grayscale
# result_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to create a binary image with white leaf and black background
# _, result_threshold = cv2.threshold(result_gray, 0, 255, cv2.THRESH_BINARY)

# # Find contours in the binary image to obtain the dimensions of the leaf
# leaf_contours, _ = cv2.findContours(result_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# leaf_contour = max(leaf_contours, key=cv2.contourArea)
# leaf_x, leaf_y, leaf_w, leaf_h = cv2.boundingRect(leaf_contour)

# # Find the two widest points on the leaf contour
# left_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmin()][0])
# right_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmax()][0])

# # Find the top and bottom points on the leaf contour
# top_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmin()][0])
# bottom_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmax()][0])

# # Draw lines for width and height
# cv2.line(image, left_point, right_point, (0, 255, 0), thickness=2)
# cv2.line(image, top_point, bottom_point, (0, 0, 255), thickness=2)

# # Calculate the width between the two widest points
# width = abs(left_point[0] - right_point[0])

# # Calculate the height between the top and bottom points
# height = abs(top_point[1] - bottom_point[1])

# # Print the dimensions of the leaf
# print("Leaf Width:", width)
# print("Leaf Height:", height)

# # Display the image with the lines and dimensions
# cv2.imshow("Leaf Image", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Leaf Width: 833
Leaf Height: 678


In [ ]:
# import cv2
# import numpy as np

# # Path to the input image
# image_path = '/Users/brandonulin/Downloads/Hognut/test_scan/test1_test.jpg'

# # Load the image
# image = cv2.imread(image_path)

# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to obtain a binary image with white edges
# _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# # Find contours in the thresholded image
# contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Find the largest contour (assumed to be the leaf)
# largest_contour = max(contours, key=cv2.contourArea)

# # Create a blank black image of the same size as the original image
# result = np.zeros_like(image)

# # Draw the largest contour on the result image as white
# cv2.drawContours(result, [largest_contour], 0, (255, 255, 255), thickness=cv2.FILLED)

# # Convert the result image to grayscale
# result_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

# # Apply thresholding to create a binary image with white leaf and black background
# _, result_threshold = cv2.threshold(result_gray, 0, 255, cv2.THRESH_BINARY)

# # Find contours in the binary image to obtain the dimensions of the leaf
# leaf_contours, _ = cv2.findContours(result_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# leaf_contour = max(leaf_contours, key=cv2.contourArea)
# leaf_x, leaf_y, leaf_w, leaf_h = cv2.boundingRect(leaf_contour)

# # Find the two widest points on the leaf contour
# left_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmin()][0])
# right_point = tuple(leaf_contour[leaf_contour[:, :, 0].argmax()][0])

# # Find the top and bottom points on the leaf contour
# top_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmin()][0])
# bottom_point = tuple(leaf_contour[leaf_contour[:, :, 1].argmax()][0])

# # Find the midpoints along the x-axis and y-axis
# midpoint_x = int((left_point[0] + right_point[0]) / 2)
# midpoint_y = int((top_point[1] + bottom_point[1]) / 2)

# # Draw lines for width, height, and vein
# cv2.line(image, left_point, right_point, (0, 255, 0), thickness=2)
# cv2.line(image, top_point, bottom_point, (0, 0, 255), thickness=2)
# cv2.line(image, (midpoint_x, top_point[1]), (midpoint_x, bottom_point[1]), (0, 0, 255), thickness=2)

# # Calculate the width between the two widest points
# width = abs(left_point[0] - right_point[0])

# # Calculate the height between the top and bottom points
# height = abs(top_point[1] - bottom_point[1])

# # Print the dimensions of the leaf
# print("Leaf Width:", width)
# print("Leaf Height:", height)

# # Display the result image with the lines and dimensions
# cv2.imshow("Leaf Image", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


Leaf Width: 693
Leaf Height: 856
